# 임베딩을 활용한 영화 추천 시스템

OpenAI 임베딩 기능을 사용하여 영화 추천 시스템을 설계하고 구축하는 과정의 세 번째 프로젝트에 오신 것을 환영합니다. 이 모듈에서는 OpenAI 기계 학습 모델의 고급 자연어 이해 기능을 활용하여 영화 제목과 설명을 분석하고 일치시켜 사용자에게 맞춤형 추천을 제공합니다.

## 프로젝트 목적

이 프로젝트는 다음과 같은 추천 시스템을 구축하도록 안내합니다.

1. **영화 데이터 처리**: 영화 제목과 설명을 콘텐츠의 본질을 포착하는 임베딩으로 변환합니다.
2. **유사성 계산**: 임베딩을 사용하여 사용자 쿼리 또는 과거 사용자 상호 작용을 기반으로 영화 간의 유사성을 찾습니다.
3. **추천 생성**: 사용자의 취향과 시청 기록에 맞는 추천 영화 목록을 제공합니다.

### Setting up API Key

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from openai import OpenAI
client = OpenAI()

embedding_model = "text-embedding-3-small"  # 사용할 텍스트 임베딩 모델

## Similarity (유사도) 측정 방법

In [3]:
toy_dataset = [
    "명량은 이순신 장군에 관한 영화다.",
    "해리포터는 마법사와 마법에 관한 영화 입니다",
    "영화 매트릭스에서 AI는 이미 가장 강력한 존재가 되어 있습니다.'"
]

In [5]:
import numpy as np

# 임베딩을 저장할 빈 리스트 생성
toy_vectors = []

# 여러 문장에 대한 임베딩 생성 요청
embeddings = client.embeddings.create(
    model=embedding_model,  # 사용할 모델 지정
    input=toy_dataset        # 임베딩을 생성할 문장들의 리스트
)

# 생성된 임베딩을 toy_vectors 리스트에 추가
for data in embeddings.data:
    toy_vectors.append(data.embedding)

np.array(toy_vectors).shape

(3, 1536)

In [6]:
user_embedding = client.embeddings.create(
    model=embedding_model,
    input="이순신 장군 영화를 추천해 주세요."
)

user_vector = user_embedding.data[0].embedding
np.array(user_vector).shape

(1536,)

내적 연산이 가능하도록 user_vector의 shape을 2차원 matrix 형태로 바꾸어 줍니다.

In [8]:
user_vector = np.array(user_vector).reshape(1, -1)
user_vector.shape

(1, 1536)

cosine 유사도 계산

In [10]:
similarities = np.dot(user_vector, np.array(toy_vectors).T)
similarities

array([[0.60100796, 0.23606571, 0.21466633]])

## 가장 유사한 벡터 추천

np.argsort()는 배열 내의 원소들을 오름차순으로 정렬했을 때의 인덱스를 반환

In [11]:
# 유사도가 높은 순서대로 원소의 인덱스를 반환
np.argsort(-similarities)

array([[0, 1, 2]], dtype=int64)

In [12]:
# 유사도가 높은 순서대로 영화의 제목 출력
[toy_dataset[id] for id in np.argsort(-similarities)[0]]

['명량은 이순신 장군에 관한 영화다.',
 '해리포터는 마법사와 마법에 관한 영화 입니다',
 "영화 매트릭스에서 AI는 이미 가장 강력한 존재가 되어 있습니다.'"]

# 대규모 데이터 세트로 확장

In [18]:
import pandas as pd

df = pd.read_excel("data/movies_5000_korean.xlsx", header=None)

df.columns = ['title', 'overview', 'kor_overview']

print(df.shape)
df.head()

(4998, 3)


,title,overview,kor_overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",Woody가 이끄는 Andy의 장난감은 Andy의 생일이 Buzz Lightyear...
1,Jumanji,When siblings Judy and Peter discover an encha...,형제 인 Judy와 Peter가 마법의 세계의 문을 열어주는 마법의 보드 게임을 발...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,가족 결혼식은 옆집 이웃과 낚시 친구 인 John과 Max 사이의 고대 불화를 통치...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","속임수를 쓰고 학대를 당하고 발을 딛고, 여자들은 숨을들이 쉬고, 애매한 ""좋은 사..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,"조지 뱅크스가 딸의 결혼식에서 회복되었을 때, 그는 그녀가 임신했다는 소식을 받았습..."


In [19]:
df.tail()

,title,overview,kor_overview
4993,The Deadly Mantis,The calving of an Arctic iceberg releases a gi...,북극 빙산의 분만은 선사 시대부터 정지 된 애니메이션에 갇힌 거대한기도 만티를 풀어...
4994,Dragonfly,A grieving doctor is being contacted by his la...,슬픔에 빠진 의사는 사망 경험 근처의 환자를 통해 늦은 아내와 연락을 취하고 있습니다.
4995,Queen of the Damned,Lestat de Lioncourt is awakened from his slumb...,Lestat de Lioncourt는 그의 잠에서 깨어났습니다. 그의 존재에 지루해...
4996,Big Bad Love,Vietnam veteran Leon Barlow is struggling as a...,베트남 베테랑 레온 바로우 (Leon Barlow)는 작가로서 어려움을 겪고 있으며...
4997,Green Dragon,A tale about Vietnamese refugees sent to an or...,베트남 난민에 대한 이야기는 캘리포니아의 캠프 펜들턴 해양 기지의 오리엔테이션 캠프...


In [20]:
# missing value 갯수 계산
df.isnull().sum()

title            0
overview        23
kor_overview    23
dtype: int64

In [21]:
# Drop missing values
df.dropna(inplace=True)
df.shape

(4975, 3)

In [22]:
# API 비용 절감을 위해 추천 시스템에 50편의 영화만 사용
df = df[:50]
df.shape

(50, 3)

In [25]:
df['kor_overview'].tail()

46    역사는 신세계의 사랑과 모험에 관한 디즈니의 서사시 애니메이션 이야기에서 영광스럽게...
47    대도시 대학교에서 문맹 퇴치 교수로 임기로 일하는 신중한 여성은 마을에 오는 현지 ...
48    L.A. 심문실에서 열린 구두 킨트는 연준에게 신화 범죄 군주 인 키저 수즈 (Ke...
49    형사 - 돌림 -Cynthia McKay (Cynthia Rothrock)는 McK...
50    레니와 그의 아내 아만다가 아기를 입양했을 때 레니는 그의 아들이 천재라는 것을 깨...
Name: kor_overview, dtype: object

In [26]:
# 임베딩을 저장할 빈 리스트 생성
toy_vectors = []

# DataFrame에서 한글 개요 열의 값을 리스트로 변환하여 임베딩 생성 요청
embeddings = client.embeddings.create(
    model=embedding_model,  # 사용할 모델 지정
    input=df['kor_overview'].values.tolist()  # 임베딩을 생성할 한글 텍스트 리스트
)

# 생성된 임베딩을 toy_vectors 리스트에 추가
for data in embeddings.data:
    toy_vectors.append(data.embedding)

In [28]:
print(len(toy_vectors[0]))

1536


In [29]:
user_request = input('어떤 영화를 원하시나요?  ')
user_embedding = client.embeddings.create(
    model=embedding_model,
    input=user_request
)

user_vector = user_embedding.data[0].embedding
user_vector = np.array(user_vector).reshape(1, -1)
user_vector.shape

어떤 영화를 원하시나요?   신세계의 사랑과 모험


(1, 1536)

벡터를 정규화(normalize)하면 벡터 간 비교를 보다 공정하고 효과적으로 수행할 수 있습니다.  정규화는 벡터의 길이(크기)를 1로 만듭니다. 이렇게 하면 모든 벡터가 동일한 스케일을 가지게 되어, 벡터 간 비교 시 크기의 영향을 받지 않습니다.

In [30]:
# user_vector 정규화
user_vector_norm = user_vector / np.linalg.norm(user_vector)

# toy_vectors 정규화
toy_vectors_norm = np.array(toy_vectors) / np.linalg.norm(toy_vectors, axis=1, keepdims=True)

# 정규화된 벡터들 간의 코사인 유사도 계산
similarities = np.dot(user_vector_norm, toy_vectors_norm.T)
similarities

array([[0.13620369, 0.18584609, 0.14163169, 0.28396919, 0.15169872,
        0.13794801, 0.24469232, 0.18766956, 0.15845267, 0.15276814,
        0.13493464, 0.11034339, 0.21602425, 0.17033837, 0.13626547,
        0.1228956 , 0.15349031, 0.11533921, 0.19123704, 0.12380528,
        0.16831806, 0.15822449, 0.14105608, 0.16557163, 0.24455175,
        0.22056524, 0.25624943, 0.11507875, 0.1784274 , 0.18962516,
        0.17209837, 0.16291691, 0.2332074 , 0.18956829, 0.16879154,
        0.15560595, 0.13548522, 0.14983788, 0.11237464, 0.07647325,
        0.23275763, 0.2242218 , 0.21892958, 0.22701337, 0.13849902,
        0.39054808, 0.08171686, 0.10006683, 0.18341132, 0.16875266]])

In [31]:
# 유사도 점수에 따라 정렬된 인덱스를 얻기 위해 np.argsort 사용
# -similarities를 사용하여 유사도가 높은 순서대로 정렬
sorted_indices = np.argsort(-similarities)[0]

# 정렬된 인덱스에 따라 상위 5개 행을 추출
top_5_rows = [df.iloc[id] for id in sorted_indices][:5]
top_5_rows

[title                                                  Pocahontas
 overview        History comes gloriously to life in Disney's e...
 kor_overview    역사는 신세계의 사랑과 모험에 관한 디즈니의 서사시 애니메이션 이야기에서 영광스럽게...
 Name: 46, dtype: object,
 title                                           Waiting to Exhale
 overview        Cheated on, mistreated and stepped on, the wom...
 kor_overview    속임수를 쓰고 학대를 당하고 발을 딛고, 여자들은 숨을들이 쉬고, 애매한 "좋은 사...
 Name: 3, dtype: object,
 title                                                Now and Then
 overview        Waxing nostalgic about the bittersweet passage...
 kor_overview    어린 시절부터 사춘기까지의 쓴맛에 대한 향수는이 부드러운 시대의 이야기, 4 명의 ...
 Name: 26, dtype: object,
 title                                                     Sabrina
 overview        An ugly duckling having undergone a remarkable...
 kor_overview    놀라운 변화를 겪은 추악한 오리는 여전히 그녀의 호감에 대한 감정을 품고 있지만, ...
 Name: 6, dtype: object,
 title                                           Leaving Las Vegas
 overview        Ben Sander